Filter any candidates that are co-located with a substation and coal candidates that overlap with their feedstock sources (we ignore any overlap between natural gas plants and their feedstock sources because pipelines are typically located underground). Additionally, standardize the columns for the distance to the feedstock source.

In [7]:
import geopandas as gpd
from pathlib import Path

electricity_siting_dir = Path.cwd() # pre-processing path

input_candidates_path = electricity_siting_dir / 'candidates'

output_candidates_path = electricity_siting_dir.parent / 'final_candidates'
output_candidates_path.mkdir(exist_ok=True)

for generator_file in input_candidates_path.glob('*gpkg'):
    gen_tech = generator_file.stem

    candidates_gdf = gpd.read_file(generator_file)

    candidates_gdf = candidates_gdf[candidates_gdf['dist_to_substation_meters'] != 0]

    if 'coal' in gen_tech:
        candidates_gdf.rename(columns={'dist_to_coal_meters': 'dist_to_feedstock_meters'}, inplace=True)
        candidates_gdf = candidates_gdf[candidates_gdf['dist_to_feedstock_meters'] != 0]

    elif 'gas' in gen_tech:
        candidates_gdf.rename(columns={'dist_to_pipeline_meters': 'dist_to_feedstock_meters'}, inplace=True)

    candidates_gdf.to_file(output_candidates_path / f'{gen_tech}.gpkg', driver='GPKG')
    print(f'Saved {gen_tech}.gpkg to {output_candidates_path}')

Saved gas_cc.gpkg to /Users/nicholaskong/Desktop/REAM_lab/hydrogen_siting/electricity_siting/final_candidates
Saved gas_cc_ccs.gpkg to /Users/nicholaskong/Desktop/REAM_lab/hydrogen_siting/electricity_siting/final_candidates
Saved coal_igcc.gpkg to /Users/nicholaskong/Desktop/REAM_lab/hydrogen_siting/electricity_siting/final_candidates
Saved coal_igcc_ccs.gpkg to /Users/nicholaskong/Desktop/REAM_lab/hydrogen_siting/electricity_siting/final_candidates
